In [1]:
!apt-get update
!apt-get install -y OpenBabel
!pip install openbabel-wheel
!pip install openfermion
!pip install pyscf openfermionpyscf
!pip install pandas
!pip install ipywidgets

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,381 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,772 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [2]:
!pip uninstall numpy -y

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4


In [1]:
!pip install numpy==1.26.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 80.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.0
    Uninstalling numpy-1.24.0:
      Successfully uninstalled numpy-1.24.0


In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
!pip install pyscf tensorflow==2.18.0 openfermion==1.7.0 cirq-core==1.4.1 numba==0.60.0

In [2]:
import numpy
import tensorflow
import cirq
import numba
import openfermion
import pyscf

print("NumPy version:", numpy.__version__)
print("TensorFlow version:", tensorflow.__version__)
print("Cirq version:", cirq.__version__)
print("Numba version:", numba.__version__)
print("OpenFermion version:", openfermion.__version__)
print("PySCF version:", pyscf.__version__)

NumPy version: 1.26.0
TensorFlow version: 2.18.0
Cirq version: 1.4.1
Numba version: 0.60.0
OpenFermion version: 1.7.0
PySCF version: 2.8.0


In [3]:
import pandas as pd
from google.colab import files
import io
from ipywidgets import widgets
import openbabel.pybel as pybel

upload = files.upload()
df = pd.read_csv(io.BytesIO(upload[list(upload.keys())[0]]), encoding='latin1')

widgets_container = widgets.VBox()
select_columns = ['name', 'symbol', 'atomic_number']

def number_box(change):
    num = change['new']
    dropdown_widgets = []
    for i in range(num):
        dropdown = widgets.Dropdown(
            options=[(f"{name} ({symbol})", symbol, atomic_number) for name, symbol, atomic_number in zip(df[select_columns[0]], df[select_columns[1]], df[select_columns[2]])],
            description=f'Atom {i + 1}',
            disabled=False
        )
        dropdown_widgets.append(dropdown)
    widgets_container.children = tuple(dropdown_widgets)

Number_atoms = widgets.IntSlider(min=1, max=12, step=1, description='Number of Atoms')
Number_atoms.observe(number_box, 'value')
display(Number_atoms, widgets_container)
number_box({'new': Number_atoms.value})

bond_widgets_container = widgets.VBox()

def bond_box(change):
    num_atoms = change['new']
    bond_widgets = []
    if num_atoms > 1:
        for i in range(num_atoms):
            for j in range(i + 1, num_atoms):
                bond_type = widgets.Dropdown(
                    options=[('No Bond', 0), ('Single Bond', 1), ('Double Bond', 2), ('Triple Bond', 3)],
                    description=f'Bond {i + 1}-{j + 1}'
                )
                bond_widgets.append(bond_type)
        bond_widgets_container.children = tuple(bond_widgets)

Number_atoms.observe(bond_box, 'value')
display(bond_widgets_container)

submit_button = widgets.Button(description='Generate SMILES')
output = widgets.Output()

generated_smiles = ""

def on_submit_button_clicked(b):
    global generated_smiles
    with output:
        output.clear_output()
        selected_atoms = [dropdown.value for dropdown in widgets_container.children]
        bond_data = []
        bond_widgets = list(bond_widgets_container.children)
        index = 0
        for i in range(len(selected_atoms)):
            for j in range(i + 1, len(selected_atoms)):
                bond_type = bond_widgets[index].value
                if bond_type != 0:
                    bond_data.append((i, j, bond_type))
                index += 1

        obMol = pybel.ob.OBMol()
        atom_ids = {}
        for i, (_, symbol, atomic_number) in enumerate(selected_atoms):
            atom = obMol.NewAtom()
            atom.SetAtomicNum(atomic_number)
            atom_ids[i] = atom

        for atom_i, atom_j, bond_type in bond_data:
            obMol.AddBond(atom_i + 1, atom_j + 1, bond_type)

        mol = pybel.Molecule(obMol)
        generated_smiles = mol.write("smi").strip()
        print(f"Generated SMILES: {generated_smiles}")

        with open("smiles_output.smi", "w") as f:
            f.write(generated_smiles)

submit_button.on_click(on_submit_button_clicked)
display(submit_button, output)

Saving periodic_table.csv to periodic_table.csv


IntSlider(value=1, description='Number of Atoms', max=12, min=1)

VBox()

VBox()

Button(description='Generate SMILES', style=ButtonStyle())

Output()

In [10]:
import openbabel.pybel as pybel
from pyscf import gto, scf

def generate_fermion_from_smiles(smiles):
    mol = pybel.readstring("smi", smiles)
    mol.make3D()

    atoms = []
    coordinates = []

    for atom in mol:
        atoms.append(atom.atomicnum)
        coordinates.append(atom.coords)

    mol_pyscf = gto.Mole()
    mol_pyscf.build(
        atom=[(str(atom), tuple(coord)) for atom, coord in zip(atoms, coordinates)],
        basis="sto-3g",
        verbose=4
    )

    mf = scf.RHF(mol_pyscf)
    mf.kernel()

    mo_coeff = mf.mo_coeff
    print("\nMolecular Orbitals:")
    for i, mo in enumerate(mo_coeff.T):
        print(f"MO {i + 1}:")
        print(mo)

    return mo_coeff, mf

def run_fermion_generation(smiles_input):
    mo_coeff, mf = generate_fermion_from_smiles(smiles_input)
    return mo_coeff, mf

In [11]:
smiles_input = generated_smiles
mo_coeff, mf = run_fermion_generation(smiles_input)

System: uname_result(system='Linux', node='5a7925c62693', release='6.1.85+', version='#1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024', machine='x86_64')  Threads 2
Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
numpy 1.26.0  scipy 1.15.2  h5py 3.13.0
Date: Wed Mar 26 06:52:42 2025
PySCF version 2.8.0
PySCF path  /usr/local/lib/python3.11/dist-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 6
[INPUT] num. electrons = 100
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 P      1.096156212603  -0.017501015372   0.021158857179 AA    2.071435031561  -0.033072125955   0.039984445177 Bohr   0.0
[INPUT]  2 Cl     1.096156212603  -1.045610493875   1.716331684618 AA    2.071435031561  -1.975917466396   3.243396822842 Bohr   0.

In [6]:
!apt-get -q update
!apt-get -q install openbabel
!pip install py3Dmol

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists...
Building dependency tree...
Reading state informat

In [12]:
import py3Dmol
import openbabel
import openbabel.pybel as pybel

def visualize_molecule_from_smiles(smiles: str):
    mol = pybel.readstring("smi", smiles)
    mol.make3D()

    if mol.OBMol.NumConformers() > 0:
        mol_3d_str = mol.write("mol")
        viewer = py3Dmol.view(width=800, height=400)
        viewer.addModel(mol_3d_str, "mol")
        viewer.setStyle({'stick': {}})
        viewer.zoomTo()

        element_colors = {
            'C': 'gray',
            'O': 'red',
            'N': 'blue',
            'H': 'white',
            'S': 'yellow',
            'P': 'orange'
        }

        for atom in mol.atoms:
            atom_name = atom.type
            atom_number = atom.atomicnum
            atom_coords = atom.coords

            if atom_name in element_colors:
                atom_color = element_colors[atom_name]
                viewer.addLabel(f"{atom_name} ({atom_number})", {
                    'position': atom_coords,
                    'backgroundColor': 'white',
                    'fontSize': 12
                })
                viewer.setStyle({'model': 0, 'atom': {'element': atom_name}}, {'stick': {'colorscheme': atom_color}})

        bond_info = []
        for bond in openbabel.OBMolBondIter(mol.OBMol):
            atom1_idx = bond.GetBeginAtomIdx() - 1
            atom2_idx = bond.GetEndAtomIdx() - 1
            bond_order = bond.GetBondOrder()

            atom1 = mol.atoms[atom1_idx]
            atom2 = mol.atoms[atom2_idx]

            atom1_coords = atom1.coords
            atom2_coords = atom2.coords

            midpoint = [(atom1_coords[0] + atom2_coords[0]) / 2,
                        (atom1_coords[1] + atom2_coords[1]) / 2,
                        (atom1_coords[2] + atom2_coords[2]) / 2]

            bond_info.append(f"Bond between {atom1.type} ({atom1.atomicnum}) and {atom2.type} ({atom2.atomicnum}) with Bond Order: {bond_order}")

            viewer.addLabel(f"Bond Order: {bond_order}", {
                'position': midpoint,
                'backgroundColor': 'yellow'
            })

        viewer.show()

        print("\nBond Information:")
        for bond in bond_info:
            print(bond)

        legend = """
        Atom Color Legend:
        - C: Carbon (Gray)
        - O: Oxygen (Red)
        - N: Nitrogen (Blue)
        - H: Hydrogen (White)
        - S: Sulfur (Yellow)
        - P: Phosphorus (Orange)
        """
        print(legend)
    else:
        print("3D structure generation failed. Please check the molecule or SMILES string.")

smiles_string = generated_smiles

visualize_molecule_from_smiles(smiles_string)

/usr/local/lib/python3.11/dist-packages/openbabel/__init__.py:26: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')


3Dmol.js failed to load for some reason. Please check your browser console for error messages.


Bond Information:
Bond between P (15) and Cl (17) with Bond Order: 1
Bond between P (15) and Cl (17) with Bond Order: 1
Bond between P (15) and Cl (17) with Bond Order: 1
Bond between P (15) and Cl (17) with Bond Order: 1
Bond between P (15) and Cl (17) with Bond Order: 1

        Atom Color Legend:
        - C: Carbon (Gray)
        - O: Oxygen (Red)
        - N: Nitrogen (Blue)
        - H: Hydrogen (White)
        - S: Sulfur (Yellow)
        - P: Phosphorus (Orange)
        


In [8]:
import openfermion
from openfermion import *
from openfermion.ops import *

In [20]:
import openfermion
from openfermion import hamiltonians
import numpy as np
import cirq

def construct_hamiltonian(mo_coeff, mf, molecule):
    num_orbitals = mo_coeff.shape[0]
    hamiltonian = openfermion.FermionOperator()
    fock_matrix = mf.get_fock()

    for i in range(num_orbitals):
        for j in range(num_orbitals):
            hamiltonian += 0.5 * fock_matrix[i, j] * openfermion.FermionOperator(((i, 1), (j, 0)))

    eri = molecule.intor('int2e')
    for i in range(num_orbitals):
        for j in range(num_orbitals):
            for k in range(num_orbitals):
                for l in range(num_orbitals):
                    hamiltonian += eri[i, j, k, l] * openfermion.FermionOperator(
                        ((i, 1), (j, 1), (k, 0), (l, 0))
                    )

    return hamiltonian


def run_fermion_generation_and_hamiltonian(smiles_input):
    mo_coeff, mf = generate_fermion_from_smiles(smiles_input)
    molecule = mf.mol
    hamiltonian = construct_hamiltonian(mo_coeff, mf, molecule)
    return hamiltonian


smiles_input = generated_smiles
hamiltonian = run_fermion_generation_and_hamiltonian(smiles_input)

print("Fermionic Hamiltonian:")
print(hamiltonian)


def jordan_wigner_transformation(hamiltonian, num_qubits):
    qubit_hamiltonian = cirq.PauliSum()
    for term, coefficient in hamiltonian.terms.items():
        pauli_string = []
        for i, spin in term:
            if spin == 1:
                pauli_string.append(cirq.X(i))
            else:
                pauli_string.append(cirq.I(i))
        qubit_hamiltonian += coefficient * cirq.PauliString(*pauli_string)
    return qubit_hamiltonian


num_qubits = 7
qubit_hamiltonian = jordan_wigner_transformation(hamiltonian, num_qubits)

print("Qubit Hamiltonian:")
print(qubit_hamiltonian)

Streaming output truncated to the last 5000 lines.
0.0005780908060557452 [53^ 50^ 38 17] +
7.331407288546106e-07 [53^ 50^ 38 18] +
7.715390342267647e-06 [53^ 50^ 38 19] +
0.0005805752341076823 [53^ 50^ 38 20] +
-5.030907850450562e-06 [53^ 50^ 38 21] +
4.803621375226862e-06 [53^ 50^ 38 22] +
-3.764921825450021e-07 [53^ 50^ 38 23] +
-0.0006902597116508028 [53^ 50^ 38 24] +
0.0006384345517330103 [53^ 50^ 38 25] +
-5.00745708263751e-05 [53^ 50^ 38 26] +
7.351601217198067e-07 [53^ 50^ 38 27] +
7.733257421677174e-06 [53^ 50^ 38 28] +
0.0005807839486833318 [53^ 50^ 38 29] +
-5.0393801148689966e-06 [53^ 50^ 38 30] +
-2.7360500210017807e-06 [53^ 50^ 38 31] +
-3.978742824451918e-06 [53^ 50^ 38 32] +
-0.0006904794114838059 [53^ 50^ 38 33] +
-0.0003627808559706523 [53^ 50^ 38 34] +
-0.0005279327361986317 [53^ 50^ 38 35] +
0.0003478098563741554 [53^ 50^ 38 36] +
0.005726736624700636 [53^ 50^ 38 37] +
0.026211284664590984 [53^ 50^ 38 38] +
0.00027001475921560863 [53^ 50^ 38 39] +
0.00283750417046914

AttributeError: 'int' object has no attribute 'dimension'